This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [62]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'yR1psyUhqQLzHFxmUEep' # removed key after completion

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [63]:
# First, import the relevant modules
import requests
import json

In [64]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&limit=1&api_key=" + API_KEY
r = requests.get(url)


In [69]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2019-12-18', 'frequency': 'daily', 'data': [['2019-12-18', 115.4, 115.9, 114.3, 115.5, None, 124036.0, 14308978.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Question 1 

In [70]:
url2 = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-12-16&end_date=2019-12-17&api_key="
afx_x_2017 = requests.get(url2 + API_KEY)

Question 2

In [75]:
afx_2017 = json.loads(afx_x_2017.text)
type(afx_2017)
print(afx_2017)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-12-16', 'end_date': '2019-12-17', 'frequency': 'daily', 'data': [['2019-12-17', 113.4, 115.6, 113.1, 115.6, None, 198741.0, 22797198.0, None, None, None], ['2019-12-16', 111.9, 113.5, 111.1, 112.9, None, 123873.0, 13933525.0, None, None, None]], 'collapse': None, 'order': None}}


Question 3

In [76]:
open_prices = [daily[1] for daily in afx_2017['dataset_data']['data'] if daily[1] is not None]
min_open = min(open_prices)
max_open = max(open_prices)
print("Opening Prices for given Period: Lowest is $%.2f and Highest is $%.2f." % (min_open, max_open))

Opening Prices for given Period: Lowest is $111.90 and Highest is $113.40.


Question 4

In [78]:
day_change = [(daily[2]-daily[3]) for daily in afx_2017['dataset_data']['data']]

print('Largest change in one day is $%.2f.' % max(day_change))

Largest change in one day is $2.50.


Question 5

In [79]:
two_day_change = []
prev_price = 0
for daily in afx_2017['dataset_data']['data']:
    if prev_price == 0:
        prev_price = daily[4]
    else:
        two_day_change.append(daily[4] - prev_price)
        prev_price = daily[4]
# min and max        
max_change = max(two_day_change)
min_change = min(two_day_change)

if abs(min_change) > max_change:
    print('Largest change in any two days based on closing price is -$%.2f.' % abs(min_change))
else:
    print('Largest change in any two days based on closing price is $%.2f.' % max_change)

Largest change in any two days based on closing price is -$2.70.


Question 6

In [81]:
tot_vol = 0
for daily in afx_2017['dataset_data']['data']:
    tot_vol += daily[6]
print('The average daily trading volume this year is %d trades.' 
      % (tot_vol/len(afx_2017['dataset_data']['data'])))

The average daily trading volume this year is 161307 trades.


Question 7

In [86]:
daily_t_volume = [daily[6] for daily in afx_2017['dataset_data']['data'] if daily[6] != None]
def median(l):
    sorted_l = sorted(l)
    len_l = len(l)
    index = (len_l-1)//2
    
    if (len_l%2):
        return sorted_l[index]
    else:
        return (sorted_l[index]+sorted_l[index+1])//2.0
median = str(median(daily_t_volume))
print('The median trading volume during 2017 was ' + median + '.')

The median trading volume during 2017 was 161307.0.
